# 4 실제 데이터를 텐서로 표현해보기
- 텐서로 실제 데이터 표현하기

## 4.1 이미지 다루기
- 픽셀의 스칼라 값은 8비트 정수 인코딩 사용

### 4.1.2 이미지 파일 로딩

In [4]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50)
# edgeitems – Number of array items in summary at beginning and end of each dimension (default = 3)
# threshold – Total number of array elements which trigger summarization rather than full repr (default = 1000)
# https://pytorch.org/docs/stable/generated/torch.set_printoptions.html

In [5]:
import imageio

img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
img_arr.shape
# 버전이 낮아서 요류메세지 뜨는 것 같음

C:\Users\shxod\AppData\Local\Temp\ipykernel_26936\1456048304.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

In [6]:
import imageio.v2 as imageio

img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
img_arr.shape
# 3차원 넘파이 객체 (너비, 높이, RGB채널)

(720, 1280, 3)

In [7]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
# (RGB채널, 높이, 너비) 순으로 차원 변경
# 텐서 복사본을 만드는게 아님, out과 img의 저장 공간 동일하며 크기와 스트라이드 정보만 변경

In [8]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
# batch는 N*C*H*W 형태의 텐서 

In [9]:
import os

data_dir = '../data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir)
            if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t
    
# batch는 N*C*H*W 형태의 텐서 

### 4.1.4 데이터 정규화
- 신경망은 입력값이 대략 0에서 1 사이거나 -1에서 1 사이일 때 훈련 성능이 가장 좋은 특징을 가짐

In [10]:
batch = batch.float()
batch /= 255.0
# 부호 없는 8비트 정수의 최댓값인 255로 나눈 것

In [11]:
# 입력 데이터의 평균과 표준 편차를 구해 각 채널 값이 표준 편차 넘지 않게 하는 방법
n_channels = batch.shape[1]
# batch에서 두 번째 값이 RGB채널을 나타냄

for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

## 4.2 3차원 이미지: 용적 데이터
- 채널 차원 뒤에 깊이(depth)차원을 가지기 때문에 N*C*D*H*W의 5차원 텐서가 됨

### 4.2.1 특수 포맷 로딩
- imageio 모듈에 있는 volread 함수 사용, 의료용 디지털 영상(DICOM) 파일을 연속된 형태로 조합해 3차원 넘파일 배열 만듦 

In [12]:
import imageio

dir_path = "../data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083"
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%14/99 files (14.1%25/99 files (25.3%37/99 files (37.4%50/99 files (50.5%62/99 files (62.6%72/99 files (72.7%84/99 files (84.8%96/99 files (97.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 34/99  (34.373/99  (73.799/99  (100.0%)


(99, 512, 512)

In [13]:
# channel 차원을 위한 공간 만들기
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)

vol.shape

torch.Size([1, 99, 512, 512])

## 4.3 테이블 데이터 표현하기
### 4.3.1 실세계 데이터셋 사용하기
- 상이한 형태의 값으로 이뤄진 실세계 데이터를 부동소수점 수로 이뤄진 텐서로 만들기

### 4.3.2 와인 데이터를 텐서로 읽어오기
##### csv 파일 읽기
- 파이썬에 내장된 csv 모듈을 사용하는 법
- 넘파이(NumPy)
- 판다스(Pandas)

##### 판다스가 제일 빠르고 메모리도 덜 사용함
##### 이번에는 넘파이 활용

In [14]:
import csv
wine_path = "../data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", 
                        skiprows=1) 
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [15]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
# next함수 : 반복가능한 객체에서 다음 요소 반환

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [16]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

#### 연속값, 순서값, 카테고리값

- 연속값(continous value) : 값 사이 순서 있음, 두 값의 차이가 의미를 지님 (kg, 마일 ...)
- 순서값(ordinal value) : 값 사이 순서 있음, 두 값의 차이가 의미 없음 (대/중/소 ...)
- 카테고리값(categorical value) : 값 사이 순서 없음, 두 값의 차이가 의미 없음 (1.커피/2.탄산/3.우유 ...), 명목 척도(nominal scale)이라고도 함

### 4.3.3 점수 표현하기

In [17]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  ...,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  ...,  0.4900,  9.5000],
         ...,
         [ 5.5000,  0.2900,  ...,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  ...,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [18]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [19]:
target = wineq[:, -1].long()
target
# '와인 색'처럼 값이 문자열로 이루어진 레이블이라면 각 문자열마다 대응하는 정수 할당해 처리 가능

tensor([6, 6,  ..., 7, 6])

### 4.3.4 원핫 인코딩
- 1부터 10까지의 값이 벡터 안의 10개의 원소에 대응하도록 정해두고서 원소 하나만 1로 설정하고 나머지는 모두 0으로 설정
- 포도 품종처럼 서로 간에 완전히 이산적인 경우라면 값 사이의 순서나 거리 개념이 없는 원핫 인코딩이 더 나음
- 원핫 인코딩은 scatter_ 메소드 사용

In [20]:
target_onehot = torch.zeros(target.shape[0], 10)

target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

#### scatter_ 메소드의 역할
##### 각각 인자의 역할은 다음과 같음
- 뒤에 오는 두 개의 인자가 따라야 하는 차원 나타냄
- 원핫으로 인코딩할 요소를 가리키는 인덱스가 들어있는 텐서
- 원핫 인코딩할 원소가 들어있는 텐서 혹은 단일 스칼라(이 경우에는 1)

In [21]:
# target_onehot이 2차원이므로 unsqueeze로 target에 추가 차원 만들어줌
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])

### 4.3.5 언제 카테고리화 할 것인가
<img src="https://user-images.githubusercontent.com/107118671/188830394-e65e52c8-2d0c-4c77-ab13-fe0b4c5a5ec9.png" width="70%" height="70%" align="left" /><br>

- 연속 데이터, 순서 데이터, 카테고리 데이터로 열을 다루는 방법

In [22]:
# 각 열의 평균과 표준 편차 구하기
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [23]:
data_var = torch.var(data, dim=0)
data_var

# dim=0은 차원 0에 대해서 축소(redution)연산이 수행됨을 지정

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [24]:
# 정규화
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  ..., -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  ...,  1.3422e-03, -8.2419e-01],
        ...,
        [-1.6054e+00,  1.1666e-01,  ..., -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  ..., -1.4882e+00,  1.0448e+00]])

### 4.3.6 임계값으로 찾기

In [25]:
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

- 고급 인덱싱(advanced indexing)이라 부르는 파이토치 기능을 통해 data 텐서를 인덱싱할 때 torch.bool 데이터 타입 사용하면 열이 True에 해당하는 행들만 접근 가능

In [26]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [27]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >- 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.85
 1 volatile acidity       0.33   0.28   0.28
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   6.39
 4 chlorides              0.05   0.05   0.05
 5 free sulfur dioxide   53.33  35.42  35.31
 6 total sulfur dioxide 170.60 141.83 138.36
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.19
 9 sulphates              0.47   0.49   0.49
10 alcohol               10.34  10.26  10.51


In [28]:
# 이산화황 총량을 임계값으로 사용
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)
# torch.lt 는 input < output 비교해서 맞으면 반환

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()
# 이산화황 총량으로 판별한 좋은 와인이 2727개

(torch.Size([4898]), torch.bool, tensor(2727))

In [29]:
actual_indexes = target > 5

actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()
# 실제로 좋은 와인이 3258개

(torch.Size([4898]), torch.bool, tensor(3258))

In [30]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches / n_predicted, n_matches / n_actual

(2018, 0.74000733406674, 0.6193984039287906)

In [31]:
# 2000여 개의 와인 맞췄음
# 2700여개 예측했으니 74%의 확률로 고품질의 와인 걸러냄
# 실제 좋은 와인은 3200여개고 그 중 2000여개를 찾았으니 61%만 구별해낸 것

## 4.4 시계열 데이터 다루기
### 4.4.1 시간 차원 더하기

In [32]:
bikes_numpy = np.loadtxt("../data/p1ch4/bike-sharing-dataset/hour-fixed.csv", 
                         dtype=np.float32, delimiter=',',skiprows=1,converters={1: lambda x: float(x[8:10])})
# converters는 2011-01-01에서 맨 끝의 01만 반환하도록 하는 것

bikes = torch.from_numpy(bikes_numpy)
bikes

# 크기가 C인 N개의 병렬 시퀀스로 표현이 가능(C는 채널, N은 차원)
# C는 이 데이터에서 열(columns)과 동일, N차원은 시간 축을 표현

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

### 4.4.2 시간 단위로 데이터 만들기

In [33]:
# 2년치 데이터셋을 쪼개서 일 단위처럼 나누기 
# N * C * L, 여기서 C에는 17개의 채널, 1을 한 시간으로 해서 L은 24

In [34]:
# 일별로 매 시간의 데이터셋을 구하기 위해 동일 텐서를 24시간 배치로 바라보는 뷰 필요
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [35]:
# 데이터를 일자, 시간, 17개의 열 세 개 축으로 만들기
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [36]:
# N * C * L 순서로 텐서 전치
daily_bikes = daily_bikes.transpose(1,2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

### 4.4.3 훈련 준비
- '날짜 상태'를 통해 연습

In [37]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:,9]
# 하루 동안의 날씨 상태

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [38]:
weather_onehot.scatter_(dim=1, index=first_day[:,9].unsqueeze(1).long() -1, value=1.0) 
# 날씨값은 1부터 4까지지만, 색인은 0에서 시작하므로 1을 빼줌

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [39]:
# cat함수 사용해 원래 데이터셋에 병합
torch.cat((bikes[:24], weather_onehot), 1)[:1]

#'날씨 상태'를 원핫 인코딩하여 데이터 열이 있는 차원(1번 차원)을 따라 병합함
# 17개 채널에다가 날씨 4개 넣었으므로 길이 21이어야 하는데 맞음

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [40]:
# daily_bikes 텐서에 대해서도 동일한 작업 할 수 있음
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])

daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [41]:
# scatter_ 해줌(텐서의 내용이 바뀜)
daily_weather_onehot.scatter(1, daily_bikes[:,9,:].long().unsqueeze(1) -1, 1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [42]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [43]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0
# 변수를 [0.0, 1.0] 사잇값으로 범위 변경

In [44]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min) / (temp_max - temp_min))
# 범위를 [0.0, 1.0]으로 매핑

In [45]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp)) / torch.std(temp))
# 모든 값에서 평균을 빼고 표준편차로 나누는 방법
# 가우시안 분포로 그리면, 샘플의 68%는 [-1.0, 1.0]에 분포하게 됨

## 4.5 텍스트 표현하기
- 자연어 처리(natural language processing, NLP)
- 순환 신경망(recurrent neural network, RNN)
- 최근엔 트랜스포머(transformer)
### 4.5.1 텍스트를 숫자로 변환하기

In [46]:
with open('../data/p1ch4/jane-austen/1342-0.txt', encoding='utf8') as f:
    text = f.read()

### 4.5.2 문자 원핫 인코딩
- 가장 단순한 형태의 인코딩은 1960년대에 나온 ASCII (128개의 문자를 128개의 정수로 인코딩)
- 넓은 범위의 문자 인코딩 기법은 유니코드(Unicode), 이 숫자를 비트로 표현하는 방식은 특정 인코딩 형식으로 제공됨
- 그 중 하나인 UTF-8은 숫자를 8비트 정수로 표현

In [47]:
lines = text.split('\n')
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [48]:
letter_t = torch.zeros(len(line), 128) # ASCII 제한인 128로 하드코딩
letter_t.shape

torch.Size([70, 128])

In [49]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0 # 방향이 있는 쌍따옴표처럼 ASCII에 유효하지 않은 문자는 여기서 버림
    letter_t[i][letter_index] = 1
    
letter_t

tensor([[1., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

### 4.5.3 모든 단어를 원핫 인코딩하기
- 임베딩(embedding)을 사용하여 단어 단위로 텍스트를 표현하는 효율적인 방법은 다음 절에서 살펴봄

In [50]:
def clean_words(input_str):
    punctuation = '.,;:"!?”“_-'
    word_list = input_str.lower().replace('\n', ' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [51]:
word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

len(word2index_dict), word2index_dict['impossible']
# word2index_dict는 단어를 키로, 정수를 값으로 가지는 dictionary

(7261, 3394)

In [52]:
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))
    
print(word_t.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
torch.Size([11, 7261])


## 4.7 연습 문제

In [53]:
# 1
import imageio.v2 as imageio

In [54]:
# 파란색
a_arr = imageio.imread('../data/p1ch4/image-dog/passport.jpg')
a_arr.shape

(797, 560, 3)

In [55]:
a = torch.from_numpy(a_arr)
a_float = a.type(torch.float)
a_out = a_float.permute(2, 0, 1)
a_out

tensor([[[ 43.,  43.,  ..., 255., 251.],
         [ 43.,  43.,  ..., 253., 255.],
         ...,
         [ 19.,  19.,  ..., 252., 255.],
         [147., 147.,  ..., 250., 255.]],

        [[ 53.,  53.,  ..., 255., 248.],
         [ 53.,  53.,  ..., 251., 255.],
         ...,
         [ 28.,  28.,  ..., 252., 253.],
         [154., 154.,  ..., 247., 255.]],

        [[115., 115.,  ..., 239., 229.],
         [115., 115.,  ..., 239., 241.],
         ...,
         [ 67.,  67.,  ..., 242., 244.],
         [180., 180.,  ..., 240., 248.]]])

In [56]:
a_mean_1 = torch.mean(a_out, dim=1)
a_mean_2 = torch.mean(a_mean_1, dim=1)
a_mean_2

tensor([ 47.6370,  57.7991, 116.9934])

In [57]:
# 빨강색
b_arr = imageio.imread('../data/p1ch4/image-dog/strawberry.jpg')
b_arr.shape

(400, 600, 3)

In [58]:
b = torch.from_numpy(b_arr)
b_float = b.type(torch.float)
b_out = b_float.permute(2, 0, 1)
b_out

tensor([[[225., 229.,  ..., 238., 229.],
         [227., 225.,  ..., 230., 233.],
         ...,
         [229., 220.,  ..., 101., 109.],
         [221., 213.,  ..., 102., 108.]],

        [[ 39.,  13.,  ..., 173., 147.],
         [ 29.,  14.,  ..., 158., 173.],
         ...,
         [113.,  53.,  ...,   8.,  23.],
         [ 73.,  17.,  ...,   9.,  23.]],

        [[ 60.,  37.,  ...,  69.,  48.],
         [ 46.,  33.,  ...,  60.,  59.],
         ...,
         [  4.,   1.,  ...,  18.,  26.],
         [  0.,   1.,  ...,  19.,  26.]]])

In [59]:
b_mean_1 = torch.mean(b_out, dim=1)
b_mean_2 = torch.mean(b_mean_1, dim=1)
b_mean_2

tensor([184.8967,  51.1075,  29.2876])

In [60]:
# 초록색
c_arr = imageio.imread('../data/p1ch4/image-dog/grass.jpg')
c_arr = c_arr[:,:,:-1] # 투명도 알파 채널까지 있어서 첫 세개 채널만 유지

c_arr.shape

(720, 960, 3)

In [61]:
c = torch.from_numpy(c_arr)
c_float = c.type(torch.float)
c_out = c_float.permute(2, 0, 1)
c_out

tensor([[[ 61.,  60.,  ...,  16.,  14.],
         [ 59.,  60.,  ...,  17.,  25.],
         ...,
         [ 86.,  80.,  ...,  83.,  78.],
         [ 91., 104.,  ...,  78.,  77.]],

        [[112., 111.,  ...,  18.,  16.],
         [110., 111.,  ...,  19.,  27.],
         ...,
         [160., 154.,  ..., 140., 135.],
         [161., 174.,  ..., 135., 134.]],

        [[ 19.,  18.,  ...,   0.,   0.],
         [ 17.,  18.,  ...,   0.,   5.],
         ...,
         [ 15.,   7.,  ...,  46.,  41.],
         [ 11.,  23.,  ...,  41.,  40.]]])

In [62]:
c_mean_1 = torch.mean(c_out, dim=1)
c_mean_2 = torch.mean(c_mean_1, dim=1)
c_mean_2

tensor([ 71.2210, 127.2252,  33.5540])

In [63]:
# 2
import torch

with open('../data/p1ch4/bike-sharing-dataset/fix_missing_hours.py', 
          encoding='utf8') as f:
    text = f.read()

In [64]:
import re

def clean_words(input_str):
    punctuation = '.,;:"!?”“_-'
    word_list = input_str.lower().replace('\n', ' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

text = re.sub(r"[^a-zA-Z0-9_]+", ' ', text)
word_list = sorted(set(clean_words(text)))
word2seq = {word:i for (i, word) in enumerate(word_list)}
word2seq

{'0': 0,
 '1': 1,
 '24': 2,
 '5': 3,
 'as': 4,
 'atemp': 5,
 'casual': 6,
 'cnt': 7,
 'copy': 8,
 'csv': 9,
 'deepcopy': 10,
 'dteday': 11,
 'elif': 12,
 'else': 13,
 'fixed': 14,
 'fixed_csv': 15,
 'fixed_file': 16,
 'for': 17,
 'holiday': 18,
 'hour': 19,
 'hour_csv': 20,
 'hour_file': 21,
 'hr': 22,
 'hum': 23,
 'if': 24,
 'import': 25,
 'in': 26,
 'instant': 27,
 'int': 28,
 'is': 29,
 'last_hour': 30,
 'last_row': 31,
 'missing_hour': 32,
 'missing_row': 33,
 'mnth': 34,
 'newline': 35,
 'none': 36,
 'open': 37,
 'pass': 38,
 'print': 39,
 'range': 40,
 'reader': 41,
 'registered': 42,
 'season': 43,
 'temp': 44,
 'this_hour': 45,
 'this_row': 46,
 'w': 47,
 'weathersit': 48,
 'weekday': 49,
 'windspeed': 50,
 'with': 51,
 'workingday': 52,
 'writer': 53,
 'writerow': 54,
 'yr': 55}

In [65]:
n_text = []
for i in text.split(' ')[:-1]:
    if i != "None":
        n_text.append(word2seq[i])
text_t = torch.tensor(n_text)
text_t
text_one_hot = torch.zeros(text_t.shape[0], len(word_list))
text_one_hot.scatter_(
    dim = 1,
    index = text_t.unsqueeze(1).long(),
    value = 1.0
)
text_one_hot

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])